In [137]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP
import os

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas as pd
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


### Gene inactivation

In [164]:
inactivation = pandas.read_csv("./cytoscape/inactivation.csv")
inactivation["blockage"] = False
inactivation

,SOURCE,TARGET,blockage
0,yugH,yugH,False
1,yvrB,samT,False
2,yvrB,metE,False
3,yvrB,yvrB,False
4,sufU,sigH,False
...,...,...,...
645,odhB,fumC,False
646,odhB,yvrB,False
647,odhB,cysI,False
648,odhB,cysJ,False


In [165]:
map = {(v["SOURCE"],v["TARGET"]):idx for idx,v in inactivation.iterrows()}

### Gene blockage

In [166]:
blockage = pandas.read_csv("./cytoscape/blockage.csv")
blockage

,SOURCE,TARGET
0,dapG,dapG
1,hom,hom
2,hom,metAA
3,hom,metI
4,cysJ,cysI
5,cysJ,sat
6,cysJ,cysH
7,cysJ,cysJ
8,fadE,fadE
9,cysI,cysI


In [167]:
for _,v in blockage.iterrows():
    edge = (v["SOURCE"],v["TARGET"])
    idx = map.get(edge,None)
    if idx is None:
        print(edge)
        continue
    inactivation.loc[idx,"blockage"] = True

('pncB', 'pncB')


### To table

In [168]:
known_depletions = [g[0].lower() + g[1:] for g in list(pd.read_csv('proteomics/known_depletions.txt')['protein'].values)]
confirmed_depletions = ['pckA','purL','ilvB','ilvE','citZ','acsA','etfA','acoC','sigF','yxbC','lutC',
                  'argJ','hpf']

In [169]:
def get_properties(gene):
    return {
        "confirmed" : True if gene in confirmed_depletions else False,
        "known" : True if gene in known_depletions else False
    }

In [170]:
dct = {}
counter = 0
for idx,row in inactivation.iterrows():
    i,j = row[["SOURCE","TARGET"]]
    if i == j:
        continue
    source_properties = {"SOURCE_"+k:v for k,v in get_properties(i).items()}
    target_properties = {"TARGET_"+k:v for k,v in get_properties(j).items()}


    tmp = {k:v for k,v in row.items()}
    tmp.update(source_properties)
    tmp.update(target_properties)
    dct[counter] = tmp
    counter += 1

In [171]:
properties = pandas.DataFrame.from_dict(dct).T

In [172]:
properties.set_index("SOURCE").to_csv("./cytoscape/network.csv")

In [173]:
inactivation[inactivation["TARGET"].str.contains("citZ")]

,SOURCE,TARGET,blockage
12,sufU,citZ,False
359,pdhC,citZ,False
384,pdhB,citZ,False
414,pdhD,citZ,False
439,pdhA,citZ,False
547,pckA,citZ,False
575,citB,citZ,False
586,citZ,citZ,False
601,icd,citZ,False


In [148]:
properties[properties["TARGET"].str.contains("citZ")]

,SOURCE,TARGET,blockage,SOURCE_confirmed,SOURCE_known,TARGET_confirmed,TARGET_known
10,sufU,citZ,False,False,False,True,True
303,pdhC,citZ,False,False,False,True,True
327,pdhB,citZ,False,False,False,True,True
356,pdhD,citZ,False,False,True,True,True
380,pdhA,citZ,False,False,False,True,True
474,pckA,citZ,False,True,True,True,True
500,citB,citZ,False,False,True,True,True
523,icd,citZ,False,False,False,True,True


In [149]:
properties[properties["TARGET"].str.contains("ilvC")]

,SOURCE,TARGET,blockage,SOURCE_confirmed,SOURCE_known,TARGET_confirmed,TARGET_known
9,sufU,ilvC,False,False,False,False,True
77,mccB,ilvC,False,False,False,False,True
81,mccA,ilvC,False,False,False,False,True
94,maeB,ilvC,False,False,False,False,True
230,trxA,ilvC,False,False,False,False,True
355,pdhD,ilvC,False,False,True,False,True
391,ilvD,ilvC,False,False,False,False,True


In [150]:
properties[properties["SOURCE"].str.contains("ilvC")]

,SOURCE,TARGET,blockage,SOURCE_confirmed,SOURCE_known,TARGET_confirmed,TARGET_known
397,ilvC,ilvD,False,False,True,False,False
398,ilvC,mccB,False,False,True,False,False
399,ilvC,mccA,False,False,True,False,False
400,ilvC,alsS,False,False,True,False,False
